# 1. Import Dependencies

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

## Dataset
This specific dataset seperates hand gestures into 4 different classes.

- stop
- onefingerup
- twofingerup
- thumbsup

The information about each hand gesture is contained into 42 features.

In [2]:
CSV_COLUMN_NAMES = np.arange(43)
CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]
CSV_COLUMN_NAMES[42] = "Species"
SPECIES = ['stop', 'onefingerup', 'twofingerup', 'thumbsup']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_0.csv")

In [3]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [4]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,Species
140,1,100,33,82,48,53,27,31,4,23,...,41,-36,27,-47,14,-39,32,-28,48,2
109,-15,63,7,50,23,30,18,17,3,18,...,34,-34,12,-33,12,-25,25,-19,36,2
162,-52,31,-37,-2,-12,-40,0,-70,1,-99,...,21,-34,37,8,44,7,41,0,37,3
22,-11,82,17,71,39,44,54,22,71,6,...,-57,-34,38,-47,17,-57,3,-65,-11,0
39,-12,101,20,89,45,63,59,39,72,21,...,-58,-39,33,-51,6,-58,-11,-64,-27,0


In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
140,1,100,33,82,48,53,27,31,4,23,...,-16,41,-36,27,-47,14,-39,32,-28,48
109,-15,63,7,50,23,30,18,17,3,18,...,-5,34,-34,12,-33,12,-25,25,-19,36
162,-52,31,-37,-2,-12,-40,0,-70,1,-99,...,4,21,-34,37,8,44,7,41,0,37
22,-11,82,17,71,39,44,54,22,71,6,...,-29,-57,-34,38,-47,17,-57,3,-65,-11
39,-12,101,20,89,45,63,59,39,72,21,...,-24,-58,-39,33,-51,6,-58,-11,-64,-27


In [6]:
train.shape  # we have 120 entires with 4 features

(159, 42)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [7]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [8]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [9]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model")

In [10]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 4 classes.
    n_classes=len(SPECIES),
    model_dir= MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Tensorflow\\workspace\\models\\my_hand_gesture_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [11]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 20.200428, step = 0
INFO:tensorflow:global_step/sec: 75.8107
INFO:tensorflow:loss = 0.30067915, step = 100 (1.319 sec)
INFO:tensorflow:global_step/sec: 97.0819
INFO:tensorflow:loss = 0.066795714, step = 200 (1.033 

## Export Model

In [12]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Tensorflow\workspace\models\my_hand_gesture_model\temp-1636740556\saved_model.pbtxt


## Load Model

In [13]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

<ConcreteFunction pruned(examples) at 0x3C4B4A60>

In [14]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert from list to tensor
examples = tf.constant(examples)

print(test)
print(feature)

       0    1   2    3   4   5   6    7   8    9  ...  32  33  34  35  36  37  \
1    -18   97  13   84  34  59  45   36  60   20  ... -16 -53 -40  24 -46  -1   
7     -9  103  24   90  49  63  61   38  69   18  ... -27 -58 -40  29 -47   1   
14     2  102  34   88  54  58  63   30  72    9  ... -30 -55 -37  39 -49  12   
16     2  109  35   91  53  56  60   27  72    3  ... -28 -59 -37  45 -50  17   
20   -14   74  13   66  36  41  52   19  69    5  ... -26 -54 -35  41 -48  19   
23    -5   94  24   83  43  54  56   30  73   13  ... -29 -57 -33  42 -48  16   
26    -1   99  29   89  50  61  61   35  74   15  ... -31 -58 -39  35 -54  10   
35   -15  100  17   88  43  63  59   41  74   22  ... -24 -60 -41  31 -54   3   
42   -35   95   0   92  30  70  50   49  68   35  ... -17 -63 -46  22 -52  -4   
51   -14   68  15   54  36  28  28    9   4   12  ...  -9  26 -37   8 -34  -3   
54   -17   71  13   55  35  28  33    9  13    9  ...  -7  29 -39   7 -30  -3   
55   -16   71  14   55  36  

In [15]:
# make predictions of all testset
predictions = predict_fn(examples=examples)
print(predictions)

{'classes': <tf.Tensor: shape=(40, 1), dtype=string, numpy=
array([[b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3']], dtype=object)>, 'all_class_ids': <tf.Tensor: shape=(40, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       

In [16]:
# print results
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {SPECIES[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {SPECIES[test_y.iloc[idx]]}")

tf.Tensor(0.9955438, shape=(), dtype=float32)
	Prediction is stop 99.55%
	Expected: stop
tf.Tensor(0.9996283, shape=(), dtype=float32)
	Prediction is stop 99.96%
	Expected: stop
tf.Tensor(0.99824095, shape=(), dtype=float32)
	Prediction is stop 99.82%
	Expected: stop
tf.Tensor(0.99712116, shape=(), dtype=float32)
	Prediction is stop 99.71%
	Expected: stop
tf.Tensor(0.9950147, shape=(), dtype=float32)
	Prediction is stop 99.50%
	Expected: stop
tf.Tensor(0.9996501, shape=(), dtype=float32)
	Prediction is stop 99.97%
	Expected: stop
tf.Tensor(0.9999385, shape=(), dtype=float32)
	Prediction is stop 99.99%
	Expected: stop
tf.Tensor(0.9999337, shape=(), dtype=float32)
	Prediction is stop 99.99%
	Expected: stop
tf.Tensor(0.9992372, shape=(), dtype=float32)
	Prediction is stop 99.92%
	Expected: stop
tf.Tensor(0.9983051, shape=(), dtype=float32)
	Prediction is onefingerup 99.83%
	Expected: onefingerup
tf.Tensor(0.99980766, shape=(), dtype=float32)
	Prediction is onefingerup 99.98%
	Expected: on

## Evaluation

In [17]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-12T19:09:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.40302s
INFO:tensorflow:Finished evaluation at 2021-11-12-19:09:22
INFO:tensorflow:Saving dict for global step 5000: accuracy = 1.0, average_loss = 0.0014474492, global_step = 5000, loss = 0.0014474492
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000

Test set accuracy: 1.000



## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [18]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = SPECIES[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES[class_id], 100 * probability))
        print(f"Expected is {expected}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "stop" (99.6%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "stop" (100.0%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
